In [50]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import sklearn
import numpy as np
from scipy.stats import boxcox

# data load
test = pd.read_csv('test.csv')
Xy = pd.read_csv('train.csv')

# anomaly detect
from sklearn.ensemble import IsolationForest
modelIsoF = IsolationForest(n_estimators=50, contamination='auto', max_features=1.0)

# all float feature
for col in Xy.loc[:, ['Depth', 'Porosity', 'Gross', 'Netpay', 'Permeability']].columns:
    modelIsoF.fit(Xy[[col]])
    Xy['anom'] = modelIsoF.predict(Xy[[col]])
    Xy = Xy[Xy['anom'] != -1]

Xy['Porosity'] = boxcox((Xy['Porosity'].apply(lambda x: 1/np.sqrt(x)).values), 0.5)
Xy['Depth'] = boxcox((Xy['Depth'].apply(lambda x: 1/np.sqrt(x)).values), 7)
Xy['Permeability'] = boxcox((Xy['Permeability'].apply(lambda x: 1/np.sqrt(x)).values), 2)
Xy['Netpay'] = boxcox((Xy['Netpay'].apply(lambda x: 1/np.sqrt(x)).values), 4)
Xy['Gross'] = boxcox((Xy['Gross'].apply(lambda x: 1/np.sqrt(x)).values), 4)

Xy['Onshore/Offshore'].value_counts()
y = Xy['Onshore/Offshore'].map({'OFFSHORE':0, 'ONSHORE':1, 'ONSHORE-OFFSHORE':2}).values
X = Xy.loc[:, ['Depth', 'Porosity', 'Gross', 'Netpay', 'Permeability']].values


test['Porosity'] = boxcox((test['Porosity'].apply(lambda x: 1/np.sqrt(x)).values), 0.5)
test['Depth'] = boxcox((test['Depth'].apply(lambda x: 1/np.sqrt(x)).values), 7)
test['Permeability'] = boxcox((test['Permeability'].apply(lambda x: 1/np.sqrt(x)).values), 2)
test['Netpay'] = boxcox((test['Netpay'].apply(lambda x: 1/np.sqrt(x)).values), 4)
test['Gross'] = boxcox((test['Gross'].apply(lambda x: 1/np.sqrt(x)).values), 4)
test = test.loc[:, ['Depth', 'Porosity', 'Gross', 'Netpay', 'Permeability']].values
#?
# select best feature
#from sklearn.feature_selection import SelectKBest
#from sklearn.feature_selection import chi2
#select = SelectKBest(chi2, k=2)
#x = select.fit(df_ctr.values, y)


i_0 = np.where(y==0)[0]
i_1 = np.where(y==1)[0]
i_2 = np.where(y==2)[0]
n_0 = len(i_0)
n_1 = len(i_1)
n_2 = len(i_2)
up_2 = np.random.choice(i_2, size=n_0, replace=True)
y = np.concatenate((np.concatenate((y[up_2], y[i_1])), y[i_0]))
X = np.vstack((np.vstack((X[up_2,:], X[i_1])), X[i_0]))



# test train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =  X, test, y, 0 # train_test_split(X, y, test_size=0.33, random_state=42) #



# standartize befor selectKBest!!!!!!!!&&&&?????????????????
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score, LeaveOneOut

model = GaussianNB()
model.fit(X_train, y_train)
from sklearn.metrics import accuracy_score, balanced_accuracy_score
# balanced_accuracy_score(y_test, model.predict(X_test))

ser = pd.Series(model.predict(X_test)).map({0:'OFFSHORE', 1:'ONSHORE', 2:'ONSHORE-OFFSHORE'})
ser.to_csv('answerbest.csv', index=False)

In [48]:

accuracy_score(y_test, model.predict(X_test))

0.8095238095238095

In [27]:
y

array([0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0])

In [28]:
i_0 = np.where(y==0)[0]
i_1 = np.where(y==1)[0]
i_2 = np.where(y==2)[0]
n_0 = len(i_0)
n_1 = len(i_1)
n_2 = len(i_2)

In [41]:
up_2 = np.random.choice(i_2, size=n_0, replace=True)
y = np.concatenate((np.concatenate((y[up_2], y[i_1])), y[i_0]))
y

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [43]:
 X = np.vstack((np.vstack((X[up_2,:], X[i_1])), X[i_0]))

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import sklearn
import numpy as np
from scipy.stats import boxcox

test = pd.read_csv('test.csv')
Xy = pd.read_csv('train.csv')
y = Xy['Onshore/Offshore'].map({'OFFSHORE':0, 'ONSHORE':1, 'ONSHORE-OFFSHORE':2}).values
X = Xy.loc[:, ['Depth', 'Porosity', 'Gross', 'Netpay', 'Permeability']].values
tst = test.loc[:, ['Depth', 'Porosity', 'Gross', 'Netpay', 'Permeability']].values

i_0 = np.where(y==0)[0]
i_1 = np.where(y==1)[0]
i_2 = np.where(y==2)[0]
n_0 = len(i_0)
n_1 = len(i_1)
n_2 = len(i_2)
up_2 = np.random.choice(i_2, size=n_0, replace=True)
y = np.concatenate((np.concatenate((y[up_2], y[i_1])), y[i_0]))
X = np.vstack((np.vstack((X[up_2,:], X[i_1])), X[i_0]))


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =  X, tst, y, 0 #train_test_split(X, y, test_size=0.33, random_state=42) #

from sklearn.ensemble import RandomForestClassifier
model_forest = RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample',
                                     criterion='entropy', max_features='sqrt', n_estimators=45,
                                     oob_score=False, verbose=0)
model_forest.fit(X_train, y_train)
#balanced_accuracy_score(y_test, model_forest.predict(X_test))


RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy',
                       max_features='sqrt', n_estimators=45)

In [5]:
from sklearn.model_selection import GridSearchCV
param={'criterion': ['gini', 'entropy'], 'max_features': ['auto', 'sqrt', 'log2'], 'n_estimators': [45, 47, 50], 'class_weight': ['balanced_subsample', 'balanced']}
grid = GridSearchCV(model_forest, param)
%time grid.fit(X_train, y_train)
model_forest = grid.best_estimator_
from sklearn.metrics import accuracy_score, balanced_accuracy_score
accuracy_score(y_test, model_forest.predict(X_test))

CPU times: user 27.2 s, sys: 13.8 ms, total: 27.2 s
Wall time: 27.4 s


0.803030303030303

In [6]:
balanced_accuracy_score(y_test, model_forest.predict(X_test))

0.7722222222222221

In [7]:
grid.best_params_

{'class_weight': 'balanced',
 'criterion': 'gini',
 'max_features': 'auto',
 'n_estimators': 50}

In [2]:
ser = pd.Series(model_forest.predict(X_test)).map({0:'OFFSHORE', 1:'ONSHORE', 2:'ONSHORE-OFFSHORE'})
ser.to_csv('answBEST.csv', index=False)